In [1]:
import pandas as pd
import re
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import apriori, association_rules
import datetime as dt
from dateutil.relativedelta import relativedelta
import os
from collections import Counter  

In [2]:
input_path = 'C:/GMG_R&D/3. 장바구니 분석/1. 한총' 
os.chdir(input_path)
path=os.getcwd()
print(path)

x = dt.datetime.now()
file_name=f'분기보고서_{x.year}년_{(x - relativedelta(months=1)).month}월/분기_{x.month}월 {x.day}일'

if not os.path.exists(file_name):
    os.makedirs(file_name) 

C:\GMG_R&D\3. 장바구니 분석\1. 한총


In [3]:
#########  필요한 함수 정의 #########
#1. 데이터셋 만들기
def dataset_make(dic_category, month_df):
    dataset = []   
    for i in range(len(month_df)) :
        sku = month_df['Model Code (v41)'][i]
        revised = sku.split(",")
        code = []
        for j in range(len(revised)) :
            code.append(dic_category[revised[j]])   
        for k in range(month_df[month_df.columns[1]][i]) :  
            dataset.append(code) 
    return dataset

#2. 주문건수
def order_2depth(new_rex, dataset):
    per_order={}
    for PRODU in new_rex['Category2D']:
        df_count=[s for s in dataset if any(PRODU in l for l in s)] 
        per_order[PRODU]=len(df_count)

    daf=pd.DataFrame.from_dict(per_order, orient="index").reset_index()
    daf_1=daf.rename({'index':'Category2D',0:"count"},axis=1)
    return daf_1



In [4]:
#카테고리 가져오기
rex= pd.read_clipboard(sep='\t')
rex=rex.astype(str)
rex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   depth1            41 non-null     object
 1   Category1D        41 non-null     object
 2   Category2D        41 non-null     object
 3   Category2D_Value  41 non-null     object
dtypes: object(4)
memory usage: 1.4+ KB


In [10]:
df= pd.read_clipboard(sep='\t')
df=df[1:]
df=df.reset_index().rename(columns={"index": "Model Code (v41)"})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40461 entries, 0 to 40460
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Model Code (v41)            40461 non-null  object
 1   Jan 1, 2022 - Jan 31, 2022  40461 non-null  int64 
 2   Feb 1, 2022 - Feb 28, 2022  40461 non-null  int64 
 3   Mar 1, 2022 - Mar 31, 2022  40461 non-null  int64 
 4   Apr 1, 2022 - Apr 30, 2022  40461 non-null  int64 
 5   May 1, 2022 - May 31, 2022  40461 non-null  int64 
 6   Jun 1, 2022 - Jun 30, 2022  40461 non-null  int64 
 7   Jul 1, 2022 - Jul 31, 2022  40461 non-null  int64 
 8   Aug 1, 2022 - Aug 31, 2022  40461 non-null  int64 
 9   Sep 1, 2022 - Sep 30, 2022  40461 non-null  int64 
 10  Oct 1, 2022 - Oct 31, 2022  40461 non-null  int64 
 11  Nov 1, 2022 - Nov 30, 2022  40461 non-null  int64 
 12  Dec 1, 2022 - Dec 31, 2022  40461 non-null  int64 
 13  Jan 1, 2023 - Jan 31, 2023  40461 non-null  in

In [11]:
new_df=[]

for y in df.columns[1:]:
    month_df = df[['Model Code (v41)',y]]
    month_df=month_df[month_df['Model Code (v41)'].str.contains(',')]
    month_df = month_df[month_df[y]!=0]
    month_df = month_df.reset_index(drop=True)
    print(y)

    #전체 SKU 하나의 리스트로
    a = month_df['Model Code (v41)'].tolist()
    b = ','.join(a) #리스트 문자열로
    c = b.split(',') #문자열 개별 문자로 리스트화 
    model = list(set(c)) #중복 제거
    ## 1. 카테고리 생성 : 부문별_카테고리 1depth_카테고리 2depth
    dic_category_2depth = {}
    for m in model :
        flag = 1
        for i in range(len(rex)):
            r = re.compile(rex.Category2D_Value[i])
            if r.search(m):
                dic_category_2depth[m] = rex.depth1[i]+'_'+rex.Category1D[i] +'_'+ rex.Category2D[i]  
                flag = 0
                break
        if flag :
            dic_category_2depth[m] = '기타'

    ## 2. 데이터셋
    dataset_2depth=dataset_make(dic_category_2depth, month_df)


    ## 4-1. 분석 : 2depth 주문건수
    to_order=order_2depth(rex, dataset_2depth)
    new_df.append(to_order)
    #to_order.sort_values(["count"]).to_csv(f'{path}/{file_name}/[분기] 동시구매_주문 건수_2depth.csv',index=False, encoding='utf-8-sig')


Jan 1, 2022 - Jan 31, 2022
Feb 1, 2022 - Feb 28, 2022
Mar 1, 2022 - Mar 31, 2022
Apr 1, 2022 - Apr 30, 2022
May 1, 2022 - May 31, 2022
Jun 1, 2022 - Jun 30, 2022
Jul 1, 2022 - Jul 31, 2022
Aug 1, 2022 - Aug 31, 2022
Sep 1, 2022 - Sep 30, 2022
Oct 1, 2022 - Oct 31, 2022
Nov 1, 2022 - Nov 30, 2022
Dec 1, 2022 - Dec 31, 2022
Jan 1, 2023 - Jan 31, 2023
Feb 1, 2023 - Feb 28, 2023


In [12]:
data=pd.DataFrame()
data["Category2D"]=""
for index, y in enumerate(range(14)): #2022년 2020,2021년은 12
    print(index, y+1)
    data=pd.merge(data,new_df[y], how='outer',on='Category2D',suffixes=[f"_{y}월", f"_{y+1}월"])
    data=data.fillna(0)
    #data.to_excel(f'{path}/{file_name}/data_P-{y}.xlsx')


#data.to_excel(f'{path}/{file_name}/[분기] 주문수_2022.xlsx',index=False)

0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14


In [8]:
data_2021=data 

In [9]:
data_2021

,Category2D,count_1월,count_2월,count_3월,count_4월,count_5월,count_6월,count_7월,count_8월,count_9월,count_10월,count_11월,count_12월
0,기타 배제,0,0,0,0,0,0,0,0,0,0,0,0
1,smartthings,0,0,0,0,0,0,0,5,114,130,285,169
2,기타 제품,0,0,0,0,0,0,0,0,0,1,0,2
3,음식,0,0,0,0,0,1,0,1,0,3,1,1
4,기타 소형가전,0,0,2,1,1,1,3,8,9,9,4,5
5,삼성 사운드바,0,0,2,1,1,0,6,4,3,2,3,5
6,삼성 오디오,41,37,27,30,33,30,42,41,50,58,58,70
7,라이프스타일 오디오,9,20,31,481,171,150,211,109,97,34,40,14
8,PC 액세서리,10,5,6,12,2,10,4,2,1,4,2,11
9,메모리/스토리지,8,8,9,0,4,1,3,15,4,2,1,3


In [13]:
data_2022=data

In [17]:
pd.merge(data_2021, data_2022, how='outer',on='Category2D',suffixes=["_2021", "_2022"]).to_excel(f'{path}/{file_name}/[분기]동시구매 주문수_2021_2023.xlsx', index=False)